## White listed files

This notebook demonstrates that some files in the DDP dataset may be suitable for analysis. 

#### Imports

In [1]:
import mne      # toolbox for analyzing and visualizing EEG data
import os       # using operating system dependent functionality (folders)
import pandas   # data analysis and manipulation
import numpy    # numerical computing (manipulating and performing operations on arrays of data)
import copy     # Can Copy and Deepcopy files so original file is untouched.
import glob
import numpy as np
import pandas as pd
from numpy.fft import fft, fftfreq
from scipy import signal

import matplotlib
import matplotlib.pyplot as plt

from mne.time_frequency.tfr import morlet
from mne.viz import plot_filter, plot_ideal_filter


from IPython.display import clear_output
import warnings

import sys

Below eegyolk is importted in it's most updated version
But the cell could be replaced by importing the stable library and then importing the modules of it.

In [2]:

import sys
sys.path.insert(0, '../eegyolk') # path to helper functions
import helper_functions as hf # library useful for eeg and erp data cleaning
#import initialization_functions #library to import data
import epod_helper
import rawf
from config import Config
from rawf import RawData

#### Load EEG files
Change your config file to change where data is coming from

In [3]:
config = Config()


In [7]:
#glob.glob(data_file_directory)

In [13]:
#eeg_file_root = "../../volume-ceph/ePodium_projectfolder"
acquired = RawData(config.get_directory('data'), config.get_directory('metadata'))

Note we assume you are looking at cnt files

In [14]:
acquired.raw.head()

,code,cnt_path,cnt_file,age_group,age_days,age_months,age_years
0,35,/volume-ceph/DDP_projectfolder/dataset/11mnd m...,035_11_jc_mmn36_slp_mmn25_slp,11,331.0,11.033333,0.919444
1,27,/volume-ceph/DDP_projectfolder/dataset/11mnd m...,027_11_jc_mmn25_wk,11,326.0,10.866667,0.905556
2,25,/volume-ceph/DDP_projectfolder/dataset/11mnd m...,025_11_mc_mmn36_wk,11,360.0,12.000000,1.000000
3,35,/volume-ceph/DDP_projectfolder/dataset/11mnd m...,035_11_jc_mmn36slp_mmn25_slp_2,11,331.0,11.033333,0.919444
4,30,/volume-ceph/DDP_projectfolder/dataset/11mnd m...,030_11_jc_mmn36_wk_mmn25_wk,11,328.0,10.933333,0.911111


Now we can make a list of all the files.

In [84]:
list_all_old_files = []
for n in range(len(acquired.raw)):
    list_all_old_files.append(acquired.raw.cnt_path.to_list()[n])

In [85]:
bad_list = []
for file in list_all_old_files:
    if check_if_numsamples_is_zero(file):
        badlist.append(file)

['/volume-ceph/DDP_projectfolder/dataset/11mnd mmn/035_11_jc_mmn36_slp_mmn25_slp.cnt',
 '/volume-ceph/DDP_projectfolder/dataset/11mnd mmn/027_11_jc_mmn25_wk.cnt',
 '/volume-ceph/DDP_projectfolder/dataset/11mnd mmn/025_11_mc_mmn36_wk.cnt',
 '/volume-ceph/DDP_projectfolder/dataset/11mnd mmn/035_11_jc_mmn36slp_mmn25_slp_2.cnt',
 '/volume-ceph/DDP_projectfolder/dataset/11mnd mmn/030_11_jc_mmn36_wk_mmn25_wk.cnt',
 '/volume-ceph/DDP_projectfolder/dataset/11mnd mmn/027_11_jc_mmn47_wk.cnt',
 '/volume-ceph/DDP_projectfolder/dataset/11mnd mmn/028_11_mc_mmn25_slp.cnt',
 '/volume-ceph/DDP_projectfolder/dataset/11mnd mmn/031_11_mc_mmn36_wk.cnt',
 '/volume-ceph/DDP_projectfolder/dataset/11mnd mmn/035_11_jc_mmn25_slp_1.cnt',
 '/volume-ceph/DDP_projectfolder/dataset/11mnd mmn/033_11_mc_mmn_25_wk.cnt',
 '/volume-ceph/DDP_projectfolder/dataset/11mnd mmn/035_11_jc_mmn25_slp_2.cnt',
 '/volume-ceph/DDP_projectfolder/dataset/11mnd mmn/036_11_mc_mmn47_slp.cnt',
 '/volume-ceph/DDP_projectfolder/dataset/11mnd 

In [10]:
def read_raw_agnostic(fname, int_arg):
    """
    For testing purposes
    """
    
    cnt_read_args = {
        'eog': 'auto',
        'data_format': int_arg,
        'date_format': 'dd/mm/yy',
        'verbose': False,
    }
    read = mne.io.read_raw_cnt(
            fname,
            data_format=int_arg,
            preload=True,
            
        )
    return read

In [45]:
def read_raw_info(fname):
    # for file in acquired.raw.cnt_path:
#      print(mne.utils.fill_doc(file))

    """
    For testing purposes
    """
    
    cnt_read_args = {
        'eog': 'auto',
        'data_format': 'int16',
        'date_format': 'dd/mm/yy',
        'verbose': False,
    }
    read = mne.io.read_raw_cnt(
            fname,
            data_format='int16',
            preload=True,
            
        )
    info = read.info
    return info

In [68]:
# what happens when we read in a cnt
mne.io.read_raw_cnt?

So we saw we want to numsamples field out of the metadata to tell us whetehr to read on int16 or int32

In [69]:
# # is it in the info?, turns out no
# info?

In [50]:
acquired.raw.cnt_path.to_list()[1]

'/volume-ceph/DDP_projectfolder/dataset/11mnd mmn/027_11_jc_mmn25_wk.cnt'

In [73]:

info = read_raw_info(acquired.raw.cnt_path.to_list()[1])
    
    #love = mne.io.read_raw_cnt(file) #, data_format='int16')
    #love.info
#     with open(file, 'rb', buffering=0) as fid:
        
#         print(fid)
raw_file = mne.io.read_raw_cnt(acquired.raw.cnt_path.to_list()[2])

Reading 0 ... 368899  =      0.000 ...   737.798 secs...


/tmp/ipykernel_3942376/1524618096.py:15: RuntimeWarning:   Could not parse meas date from the header. Setting to None.
  read = mne.io.read_raw_cnt(
/tmp/ipykernel_3942376/278839607.py:8: RuntimeWarning:   Could not parse meas date from the header. Setting to None.
  raw_file = mne.io.read_raw_cnt(acquired.raw.cnt_path.to_list()[2])
/tmp/ipykernel_3942376/278839607.py:8: RuntimeWarning: Could not define the number of bytes automatically. Defaulting to 2.
  raw_file = mne.io.read_raw_cnt(acquired.raw.cnt_path.to_list()[2])


In [81]:
with open(acquired.raw.cnt_path.to_list()[1],'r', encoding='utf-8') as f:

    for line in f:

        l=line.strip()

        print(l)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xbd in position 13: invalid start byte

In [ ]:
raw_df32 = read_on_32.to_data_frame()
raw_df32.head()

In [ ]:
read_on_16 = read_raw_agnostic(paths_df.cnt_path[n], 'int16')
raw_df16 = read_on_16.to_data_frame()
raw_df16.head()

In [ ]:
# The big difference

In [ ]:
(raw_df32 -raw_df16).sum().sum()

### Youch!

So here we saw an example of how reading the same file on int32 and int16 will give us dramatically different results. We must know which we have when we use cnt files.

In [ ]:
4*8